# Data Gathering

In [1]:
%matplotlib widget

In [2]:
from utils.io import gen_id_dated, gdf_viewer, gdf_geom, gdf_merger, na_col_drop, na_line_drop
import re, os
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import datetime as dtm
import matplotlib.pyplot as plt
from definitions import ROOT_DIR

In [3]:
def gdf_filter(data, position=True,id_col='ID', bypass_col=[], dist_crit=2, rapp_val=2, verbose=False):
    """
    filter a dataframe of duplicate values (considering ID and/or position)
    """
    
    drop_idx = []
    check_idx = []
    id_list = []
    cols = data.columns.to_list()

    for i in range(len(data)):
        uid = data.loc[i,id_col]
        tmp = data[data[id_col]== f"{uid}"]
        
        if position: # use position XY
            pos_1 = Point(data.loc[i,'X'], data.loc[i,'Y'])

            if uid not in id_list and len(tmp) >=2:
                id_list.append(uid)

                for j in tmp.index: # retrieve duplicates ID index
                    if j != i:
                        pos_2 = Point(data.loc[j, 'X'], data.loc[j, 'Y'])
                        dist = pos_1.distance(pos_2)
                        
                        """if re.search('int|float', gdf[i + '_x'].dtype.name):
                            med = np.mean([gdf[i + '_x'].median(skipna=True), gdf[i + '_y'].median(skipna=True)])
                            xgap = abs(gdf.loc[j, i + '_x'] - med) # gap between the value and median
                            ygap = abs(gdf.loc[j, i + '_y'] - med)
                        """

                        if dist <= dist_crit:
                            start = data.columns.to_list().index('Y')+1
                            if j not in drop_idx:
                                for c in range(start, len(data.columns)):
                                    if cols[c] not in bypass_col:
                                        # print('1-', cols[c])
                                        if data.iloc[i,c] == data.iloc[j,c]: # all values (str, numeric)
                                            same = True

                                        elif data.iloc[i,c] != data.iloc[j,c] and re.search('int|float', data[cols[c]].dtype.name):
                                            """
                                            if xgap < ygap:
                                                gdf.loc[j, i] = gdf.loc[j, i + '_x']
                                                gdf.loc[j, i + '_x'] = np.nan
                                                gdf.loc[j, i + '_y'] = np.nan
                                            else:
                                                gdf.loc[j, i] = gdf.loc[j, i + '_y']
                                                gdf.loc[j, i + '_x'] = np.nan
                                                gdf.loc[j, i + '_y'] = np.nan
                                            """
                                            
                                            rapp = max(data.iloc[i,c], data.iloc[j,c])/min(data.iloc[i,c], data.iloc[j,c])
                                            if rapp <= rapp_val:
                                                same = True
                                            else:
                                                same = False
                                                if j not in check_idx: check_idx.append(j)
                                                if verbose: print(f"check [{j},{cols[c]}]")
                                        else : # str values
                                            same = False
                                            if j not in check_idx: check_idx.append(j)
                                            if verbose: print(f"check [{j},{cols[c]}]")
                                        
                                if same: drop_idx.append(j) # same objet -> most be dropped
                                    
        else: # without position XY, use ID
            if uid not in id_list and len(tmp) >=2:
                id_list.append(uid)

                for j in tmp.index:
                    if j != i:
                        start = data.columns.to_list().index('Y')+1
                        if j not in drop_idx:
                            for c in range(start, len(data.columns)):
                                if cols[c] not in bypass_col:
                                    # print('2-',cols[c])
                                    if data.iloc[i,c] == data.iloc[j,c]:
                                        same = True

                                    elif data.iloc[i,c] != data.iloc[j,c] and re.search('int|float', data[cols[c]].dtype.name):
                                        rapp = max(data.iloc[i,c], data.iloc[j,c])/min(data.iloc[i,c], data.iloc[j,c])
                                        if rapp <= rapp_val:
                                            same = True
                                        else:
                                            same = False
                                            if j not in check_idx: check_idx.append(j)
                                            if verbose: print(f"check [{j},{cols[c]}]")
                                    else :
                                        same = False
                                        if j not in check_idx: check_idx.append(j)
                                        if verbose: print(f"check [{j},{cols[c]}]")

                            if same: drop_idx.append(j) # same objet -> most be dropped
        
            
    check_data = data.loc[check_idx, :]
    if len(check_data) >0:
        print(f"some data must be checked , look at indices : {check_idx}")
        
    data.drop(index=drop_idx, inplace=True)
    data.reset_index(drop=True,inplace=True)
    print(f"Rows : {data.shape[0]} ; Columns : {data.shape[1]} ; Unique on '{id_col}' : {len(set(data[id_col]))} ; ")
    
    return data, check_data

In [4]:
def files_search(work_dir, files_dict, prefix='', skip=None, details=False):   
    
    if skip is None:
        skip = "we don't want to skip a word"
        
    for k in files_dict.keys():
        tmp_list = []
        for p,d,f in os.walk(work_dir):
            for x in f:
                add = False
                if re.search(prefix,x,re.I) and not re.search(skip,x,re.I):
                    add = True
                    i = str(f'{p}/{x}')
                else:
                    add = False
                    i=''
                    
                if re.search(k,i,re.I) and add:
                    tmp_list.append(i)
        tmp_list.sort()
        files_dict.update({k:tmp_list})

    for k,v in files_dict.items():
        print(k,' \t: ',len(v))
    
    if details: # Look filenames
        which = files_dict.keys()

        for w in which:
            print('\n+++++++++++++++++')
            print(f'+  {w.upper()}\t+ ')
            print('+++++++++++++++++')
            [print(i, '-', x) for i, x in enumerate(files_dict[w], 0)]    


In [5]:
def check_col(data):
    cols_idx = []
    
    class DoubledColumns(Exception):
        """Merging process doubled column(s) still remain. Check and drop them before continue"""
        pass
    
    for i in range(len(data.columns)):
        if re.search('_x|_y', list(data.columns)[i]):
            cols_idx.append(i)
    
    if len(cols_idx) != 0 :
        raise DoubledColumns(f'Merging process doubled column(s) still remain.'
                             f'\nCheck and drop them before continue ! Doubled columns position {cols_idx}')

## Files reading

In [6]:
work_dir=ROOT_DIR+'/CF_data/Result_traitem/'

In [7]:
# create my dictionary structure to retrieve good files (Keynames !!!)
files_dict={'Borehole':0,'Piezometer':0,'Litho':0, 'Trench':0,'Equipm':0,
        'Measure':0,'Sample':0,'Analysis':0,'facility':0}

In [8]:
files_search(work_dir, files_dict, prefix='', skip='source')

Borehole  	:  5
Piezometer  	:  14
Litho  	:  6
Trench  	:  1
Equipm  	:  2
Measure  	:  6
Sample  	:  26
Analysis  	:  20
facility  	:  4


In [9]:
how=['inner', 'outer', 'left', 'right']

In [10]:
f = False # not plot df with gdf_view (only stats)
t = True

# Boreholes

Some corrections todo in 'data organization':
- correct extraction in the file 2 -> Samples
- file 4 and file 5 are the same in result (check it)
- try to concatenate file 1 with piezo (if possible because no position)
- check processing for 'refus and 'type_refus' (every object)

In [11]:
# keys: Borehole','Piezometer','Litho', 'Trench','Equipm','Measure','Sample','Analysis','facility'
files_dict['Borehole']

['/home/su530201/PycharmProjects/GSDMA/CF_data/Result_traitem/Forage_Pilote/leve_Z_elect_pos_Boreholes.csv',
 '/home/su530201/PycharmProjects/GSDMA/CF_data/Result_traitem/Prof_contact_sol_forage/Feuil1_Boreholes.csv',
 '/home/su530201/PycharmProjects/GSDMA/CF_data/Result_traitem/database_Memoris3/Profils_sol_Boreholes.csv',
 '/home/su530201/PycharmProjects/GSDMA/CF_data/Result_traitem/donnees_terrain_2019/Donnees_forage_Boreholes.csv',
 '/home/su530201/PycharmProjects/GSDMA/CF_data/Result_traitem/profils_sols_donnees_forages/donnees_forage_Boreholes.csv']

In [12]:
key='Borehole'
coi=['ID','ID_date','X','Y','Z','Type','Long_for','Diam_for','Refus'] #columns of interest
print(len(files_dict[key]), 'files')

5 files


In [13]:
df1 = pd.read_csv(files_dict[key][1], delimiter=',')
df2 = pd.read_csv(files_dict[key][3], delimiter=',')

if 'X' in list(df1.columns): print('df1 - Position data')
else: print('df1 - No position data')
if 'X' in list(df2.columns): print('df2 - Position data')
else: print('df2 - No position data')

#gdf_view = True
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

df1 - Position data
df2 - Position data
Rows : 8, columns : 6, Unique on 'ID': 8


interactive(children=(IntSlider(value=3, description='rows', max=8, min=3, readout=False), IntSlider(value=6, …

Rows : 16, columns : 18, Unique on 'ID': 16


interactive(children=(IntSlider(value=3, description='rows', max=16, min=3, readout=False), IntSlider(value=12…

(None, None)

In [14]:
df2.ID=df2.ID.apply(lambda x: 'F'+x)

In [15]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(mdf) # check if columns with '..._x' or '..._y' are still present and raise an error

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanme


Columns dropped :['Long_for_x', 'X_x', 'Y_x', 'Z_x', 'Type_x', 'X_y', 'Y_y', 'Z_y', 'Type_y', 'Long_for_y', 'Diam_ext_pz', 'Diam_int_pz', 'Long_pz', 'Crep_long']



In [16]:
gdf_viewer(mdf, rows=3, un_val='ID', view=t)

Rows : 17, columns : 14, Unique on 'ID': 17


interactive(children=(IntSlider(value=3, description='rows', max=17, min=3, readout=False), IntSlider(value=12…

In [17]:
boreholes = mdf.copy() #saving

In [20]:
df1 = pd.read_csv(files_dict[key][2], delimiter=',')
df2 = pd.read_csv(files_dict[key][4], delimiter=',')

if 'X' in list(df1.columns): print('df1 - Position data')
else: print('df1 - No position data')
if 'X' in list(df2.columns): print('df2 - Position data')
else: print('df2 - No position data')

#gdf_view = True
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

df1 - No position data
df2 - Position data
Rows : 826, columns : 6, Unique on 'ID': 172


interactive(children=(IntSlider(value=3, description='rows', max=826, min=3, readout=False), IntSlider(value=6…

Rows : 13, columns : 13, Unique on 'ID': 13


interactive(children=(IntSlider(value=3, description='rows', max=13, min=3, readout=False), IntSlider(value=12…

(None, None)

In [21]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(mdf)


Columns dropped :['ID_date_x', 'Date_ouv_x', 'Type_x', 'Societe_x', 'Method_x', 'ID_date_y', 'Date_ouv_y', 'Method_y', 'Societe_y', 'Type_y']



In [22]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=f)

Rows : 839, columns : 13, Unique on 'ID': 185


#### Last merging

In [23]:
boreholes, err_df=gdf_merger(boreholes, mdf, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(boreholes)

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanme

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanme

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanme

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanme

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanme

...

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanme

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanme

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanme


Columns dropped :['X_x', 'Y_x', 'Z_x', 'Diam_for_x', 'Refus_x', 'Type_refus_x', 'Long_for_x', 'Type_x', 'X_y', 'Y_y', 'Z_y', 'Long_for_y', 'Diam_for_y', 'Refus_y', 'Type_refus_y', 'Type_y']

Ambiguous values in both columns compared, change it manually !
Columns ['ID', 'ID_date_x', 'ID_date_y', 'Societe_x', 'Societe_y', 'Date_ouv_x', 'Date_ouv_y'] must be dropped manually !
error file created in 'tmp_files/merging_error_log(boreholes-mdf).csv'


DoubledColumns: Merging process doubled column(s) still remain.
Check and drop them before continue ! Doubled columns position [4, 5, 7, 10, 11, 12]

In [24]:
boreholes.columns[[4, 5, 7, 10, 11, 12]]

Index(['ID_date_x', 'Date_ouv_x', 'Societe_x', 'ID_date_y', 'Societe_y',
       'Date_ouv_y'],
      dtype='object')

In [25]:
gdf_viewer(boreholes, rows=3, un_val='ID', view=t)

Rows : 852, columns : 21, Unique on 'ID': 198


interactive(children=(IntSlider(value=3, description='rows', max=852, min=3, readout=False), IntSlider(value=1…

In [24]:
# not the same

In [26]:
err_df

,ID,ID_date_x,ID_date_y,Societe_x,Societe_y,Date_ouv_x,Date_ouv_y
0,F205,2019-205,2010-F205,ECOPLANNING sprl,SITEREM,2019-10-07,2010-03-26
1,F205,2019-205,2010-F205,ECOPLANNING sprl,SITEREM,2019-10-07,2010-03-26
2,F212,2019-212,2010-F212,ECOPLANNING sprl,SITEREM,2019-10-07,2010-05-10
3,F212,2019-212,2010-F212,ECOPLANNING sprl,SITEREM,2019-10-07,2010-05-10
4,F212,2019-212,2010-F212,ECOPLANNING sprl,SITEREM,2019-10-07,2010-05-10
5,F212,2019-212,2010-F212,ECOPLANNING sprl,SITEREM,2019-10-07,2010-05-10
6,F212,2019-212,2010-F212,ECOPLANNING sprl,SITEREM,2019-10-07,2010-05-10
7,F212,2019-212,2010-F212,ECOPLANNING sprl,SITEREM,2019-10-07,2010-05-10
8,F212,2019-212,2010-F212,ECOPLANNING sprl,SITEREM,2019-10-07,2010-05-10
9,F217,2019-217,2010-F217,ECOPLANNING sprl,SBS Environnement,2019-10-07,2010-03-02


# Piezometers

Some corrections todo in 'data organization':
- file 0 and file 1 are the same in result (check it)
- file 6 doesn't have 'Type' column
- file 14 contains 'Equipment' data
- files 15 and 16 are the same
- file 17 contains 'Z' but no need 

In [ ]:
key='Piezometer'
coi=['ID','ID_date','X','Y','Z','Type','Long_for','Diam_for','Refus'] #columns of interest
print(len(files_dict[key]), 'files')

In [ ]:
df1 = pd.read_csv(files_dict[key][0], delimiter=',')
df2 = pd.read_csv(files_dict[key][2], delimiter=',')

if 'X' in list(df1.columns): print('df1 - Position data')
else: print('df1 - No position data')
if 'X' in list(df2.columns): print('df2 - Position data')
else: print('df2 - No position data')

#gdf_view = True
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df2, check = gdf_filter(df2, position=True, id_col='ID', bypass_col=['Refus'], dist_crit=2, rapp_val=2)
#gdf_viewer(df2, rows=5, un_val='ID', view=f)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(mdf)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=f)

In [ ]:
piezometers = mdf.copy() #saving

In [ ]:
df1 = pd.read_csv(files_dict[key][1], delimiter=',')
df2 = pd.read_csv(files_dict[key][3], delimiter=',')

if 'X' in list(df1.columns): print('df1 - Position data')
else: print('df1 - No position data')
if 'X' in list(df2.columns): print('df2 - Position data')
else: print('df2 - No position data')

#gdf_view = True
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(mdf)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=f)

In [ ]:
piezometers, err_df=gdf_merger(piezometers, mdf, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(piezometers)
gdf_viewer(piezometers, rows=3, cols=13, un_val='ID', view=f)

In [ ]:
df1 = pd.read_csv(files_dict[key][6], delimiter=',')
df2 = pd.read_csv(files_dict[key][5], delimiter=',')

if 'X' in list(df1.columns): print('df1 - Position data')
else: print('df1 - No position data')
if 'X' in list(df2.columns): print('df2 - Position data')
else: print('df2 - No position data')

#gdf_view = True
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1, check = gdf_filter(df1, position=True, id_col='ID', bypass_col=['Refus'], dist_crit=2, rapp_val=2)
gdf_viewer(df2, un_val='ID', view=f)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(mdf)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=f)

In [ ]:
piezometers, err_df=gdf_merger(piezometers, mdf, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(piezometers)

In [ ]:
gdf_viewer(piezometers, rows=3, cols=13, un_val='ID', view=f)

In [ ]:
df1 = pd.read_csv(files_dict[key][8], delimiter=',')
df2 = pd.read_csv(files_dict[key][9], delimiter=',')

if 'X' in list(df1.columns): print('df1 - Position data')
else: print('df1 - No position data')
if 'X' in list(df2.columns): print('df2 - Position data')
else: print('df2 - No position data')

#gdf_view = True
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(mdf)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=f)

In [ ]:
piezometers, err_df=gdf_merger(piezometers, mdf, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals(), debug=False)
check_col(piezometers)

In [ ]:
gdf_viewer(piezometers, rows=3, cols=13, un_val='ID', view=f)

In [ ]:
df1 = pd.read_csv(files_dict[key][10], delimiter=',')
df2 = pd.read_csv(files_dict[key][11], delimiter=',')

if 'X' in list(df1.columns): print('df1 - Position data')
else: print('df1 - No position data')
if 'X' in list(df2.columns): print('df2 - Position data')
else: print('df2 - No position data')

#gdf_view = True
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df2['ID'] = df2['ID'].astype('object')

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(mdf)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=f)

In [ ]:
piezometers, err_df=gdf_merger(piezometers, mdf, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())#, debug=True)
check_col(piezometers)

In [ ]:
data = piezometers.copy()
for i in data.index:
    x = data.loc[i, 'Terrain_x']
    y = data.loc[i, 'Terrain_y']
    
    if not isinstance(x, float):
        x = str(x[:3]).lower()
    if not isinstance(y, float):
        y = str(y[:3]).lower()
        
    if x == y:
        if x == 'all': 
            data.loc[i, 'Terrain'] = 'Alluvions'
        elif x == 'rem':
            data.loc[i, 'Terrain'] = 'Remblais'
data.drop(columns=['Terrain_x', 'Terrain_y'], inplace=True)
piezometers = data.copy()

In [ ]:
data = piezometers.copy()
for i in data.index:
    x = data.loc[i, 'Terrain']
    
    if not pd.isnull(x):
        x = str(x[:3]).lower()
        
    if x == 'all': 
        data.loc[i, 'Terrain'] = 'Alluvions'
    elif x == 'rem':
        data.loc[i, 'Terrain'] = 'Remblais'
piezometers = data.copy()

In [ ]:
gdf_viewer(piezometers, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
df1 = pd.read_csv(files_dict[key][12], delimiter=',')
df2 = pd.read_csv(files_dict[key][13], delimiter=',')

if 'X' in list(df1.columns): print('df1 - Position data')
else: print('df1 - No position data')
if 'X' in list(df2.columns): print('df2 - Position data')
else: print('df2 - No position data')

#gdf_view = True
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(mdf)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
# temporary save to come back as 'ctrl-Z'
tmp_save = piezometers.copy()
piezometers = tmp_save.copy()

#### Last merging

In [ ]:
piezometers, err_df=gdf_merger(piezometers, mdf, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(piezometers)

In [ ]:
data = piezometers.copy()
for i in data.index:
    x = data.loc[i, 'Terrain']
    
    if not pd.isnull(x):
        x = str(x[:3]).lower()
        
    if x == 'all': 
        data.loc[i, 'Terrain'] = 'Alluvions'
    elif x == 'rem':
        data.loc[i, 'Terrain'] = 'Remblais'
        
piezometers = data.copy()

In [ ]:
gdf_viewer(piezometers, rows=3, cols=13, un_val='ID', view=t)

There are some special duplicates (different ID but same position)

- piezometers.query("ID=='P2' or ID=='P2M'")

###### Checking

# Equipments

In [ ]:
key='Equipm'
coi=['ID','ID_date','X','Y','Z','Type','Long_for','Diam_for','Refus'] #columns of interest
print(len(files_dict[key]), 'files')

In [ ]:
df1 = pd.read_csv(files_dict[key][0], delimiter=',')
df2 = pd.read_csv(files_dict[key][1], delimiter=',')

if 'X' in list(df1.columns): print('df1 - Position data')
else: print('df1 - No position data')
if 'X' in list(df2.columns): print('df2 - Position data')
else: print('df2 - No position data')

#gdf_view = True
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1['ID'] = df1['ID'].astype('object')

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(mdf)

In [ ]:
mdf['ID'] = mdf['ID'].astype('object')
equipments = mdf

In [ ]:
gdf_viewer(equipments, rows=3, cols=13, un_val='ID', view=t)

# Lithologies

Some corrections todo in 'data organization':

In [ ]:
key='Litho'
coi=['ID','X','Y','Z','Litho_top','Litho_base','Description'] #columns of interest
print(len(files_dict[key]), 'files')

In [ ]:
df1 = pd.read_csv(files_dict[key][0], delimiter=',')
df2 = pd.read_csv(files_dict[key][1], delimiter=',')

if 'X' in list(df1.columns): print('df1 - Position data')
else: print('df1 - No position data')
if 'X' in list(df2.columns): print('df2 - Position data')
else: print('df2 - No position data')

gdf_view = True
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
diff = list(set(df1['ID'])^set(df2['ID']))
print(diff, len(diff), len(set(test['ID'])))

In [ ]:
df1['Litho_base'] = df1['Litho_base'].apply(lambda v : re.sub(',','.',v) if not pd.isnull(v) else v)

In [ ]:
df1['Litho_base'] = df1['Litho_base'].astype('float')

In [ ]:
test = df1.merge(df2, how='outer', on='ID')#.query("(Litho_top_x == Litho_top_y) and (Litho_base_x == Litho_base_y)")
gdf_viewer(test, cols=13)

In [ ]:
test.query('(Litho_top_x==0.0) and (Litho_base_y==4.3)')

In [ ]:
df1['Litho_base'] = df1['Litho_base'].apply(lambda v : re.sub(',','.',v) if not pd.isnull(v) else v)
df1['Litho_base'] = df1['Litho_base'].astype('float')

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
check_col(mdf)

In [ ]:
gdf_viewer(mdf, rows=10, cols=13, un_val='ID', view=t)

In [ ]:
data = mdf.copy()
col = 'Terrain'

id_list = []
keep_idx = []

for i in data.index:
    _id = data.loc[i,'ID']
    if re.search("'", _id): 
        _id = f"`{_id}`" # it doesn't work 
    else:
        _id = f"{_id}"
    
    if _id not in id_list:
        id_list.append(_id)
        tmp = data[data['ID']==f"{_id}"]

        if len(tmp) < 2 and len(tmp) > 0:
            keep_idx = keep_idx + list(tmp.index)
        else:
            tmp = tmp[tmp[col]==tmp[col]]
            keep_idx = keep_idx + list(tmp.index)
print(keep_idx)
    
data = data.loc[keep_idx,:]
data.drop(columns=[col+'_x', col+'_y'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
mdf = data.copy()

In [ ]:
gdf_viewer(mdf, rows=10, cols=13, un_val='ID', view=t)

In [ ]:
lithologies = mdf.copy()

In [ ]:
df1 = pd.read_csv(files_dict[key][2], delimiter=',')
df2 = pd.read_csv(files_dict[key][3], delimiter=',')

if 'X' in list(df1.columns): print('df1 - Position data')
else: print('df1 - No position data')
if 'X' in list(df2.columns): print('df2 - Position data')
else: print('df2 - No position data')
    
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1.drop(columns=['Societe','Description'], inplace=True)

In [ ]:
df1['X'] = df1['X'].apply(lambda v : re.sub(',','.',v) if not pd.isnull(v) else v)
df1['Y'] = df1['Y'].apply(lambda v : re.sub(',','.',v) if not pd.isnull(v) else v)
df1['Ep_remb'] = df1['Ep_remb'].apply(lambda v : re.sub(',','.',v) if not pd.isnull(v) else v)

In [ ]:
df1['Type'] = df1['Type'].astype('object')
df1[['X','Y','Ep_remb']] = df1[['X','Y','Ep_remb']].astype('float')

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=f)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
# temporary save to come back as 'ctrl-Z'
tmp_save = lithologies.copy()
lithologies = tmp_save.copy()

In [ ]:
df1.info()

In [ ]:
lithologies, err_df=gdf_merger(lithologies, mdf, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals(), debug=t)
check_col(lithologies)

In [ ]:
lithologies[lithologies.columns[[18,21]]]

In [ ]:
df1 = pd.read_csv(files_dict[key][4], delimiter=',')
df2 = pd.read_csv(files_dict[key][5], delimiter=',')

if 'X' in list(df1.columns): print('df1 - Position data')
else: print('df1 - No position data')
if 'X' in list(df2.columns): print('df2 - Position data')
else: print('df2 - No position data')
    
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals())
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=f)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
lithologies, err_df=gdf_merger(lithologies, mdf, how=how[1], col='ID', dist_max=2, non_na=1, scope=globals(), debug=False)
check_col(lithologies)

In [ ]:
gdf_viewer(lithologies[lithologies.columns[[18,21,25,30,32,33]]], rows=3)

In [ ]:
gdf_viewer(lithologies, rows=3, cols=13, un_val='ID', view=t)

# Samples

Some corrections todo in 'data organization':
- file 0 and file 1 are the same in result (check it)

# Measures

Some corrections todo in 'data organization':
- file 0 and file 1 are the same in result (check it)

# Analysis

Some corrections todo in 'data organization':
- file 0 and file 1 are the same in result (check it)